<a href="https://colab.research.google.com/github/Erklaerbaer123/Budgeting/blob/master/TransformerNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TransformerNet

## Setup

In [ ]:
import PIL as pil


import matplotlib.pyplot as plt
import tensorflow as tf

#Import torch
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.tensorboard import SummaryWriter
from torch import nn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Tensorboard

In [ ]:
# Tensorboard

%load_ext tensorboard

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
import os
LOG_DIR = 'runs'
os.makedirs(LOG_DIR, exist_ok=True)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
%tensorboard --logdir runs

In [ ]:
writer = SummaryWriter("runs")

## Model

### Cifar Dataset

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
     ])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


### COCO Dataset



In [ ]:
from pycocotools.coco import COCO
import requests
from io import BytesIO

coco = COCO(annotation_file="/content/drive/My Drive/Datasets/coco/annotations/instances_train2017.json")

In [ ]:
#cats = coco.

In [ ]:
allIds = coco.getImgIds()

len(allIds)

In [ ]:
#catIds = coco.getCatIds(catNms=["zebra"])
annIds = coco.getAnnIds(imgIds=[144941])

anns = coco.loadAnns(annIds)

cat = coco.loadCats(anns[0]["category_id"])



In [ ]:
img = coco.loadImgs(ids=144941)

response = requests.get(img[0]["coco_url"])
img = pil.Image.open(BytesIO(response.content))

plt.axis("off")
plt.imshow(img)
#plt.show()

coco.showAnns(anns, draw_bbox=True)

### Model


In [ ]:
resnet50 = models.resnet50(pretrained=True, progress=True)

In [ ]:
class transformer (nn.Module):
  def __init__(self, hidden_dim):
    super().__init__()
    self.backbone = nn.Sequential(*list(resnet50.children())[:-2])
    self.conv = nn.Conv2d(2048, 256, 1)
    self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))
  def forward(self, inputs):
    x = self.backbone(inputs)
    h = self.conv(x)
    print(h.size())
    return h

In [ ]:
class DETR(nn.Module):
  def __init__(self, num_classes, hidden_dim, nheads, num_encoder_layers, num_decoder_layers):
    super().__init__() # We take only convolutional layers from ResNet-50 model
    self.backbone = nn.Sequential(*list(resnet50.children())[:-2]) 
    self.conv = nn.Conv2d(2048, hidden_dim, 1)
    self.transformer = nn.Transformer(hidden_dim, nheads, num_encoder_layers, num_decoder_layers)
    self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
    self.linear_bbox = nn.Linear(hidden_dim, 4)
    self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))
    self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
    self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))

  def forward(self, inputs):
    x = self.backbone(inputs)
    print(x.size())
    h = self.conv(x)
    print(h.size())
    H, W = h.shape[-2:]
    print("h.flatten:", h.flatten(2).permute(2, 0, 1).size())
    print("H: ", H)
    print("W: ", W)
    col1 = self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1)
    print("col_embed_normal:", self.col_embed.size())
    print("col_embed:", self.col_embed[:W].size())
    print("col1: ", col1.size())
    print("col1_complete: ", self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1).size())
    row1 = self.row_embed[:H].unsqueeze(1).repeat(1, W, 1)
    pos = torch.cat([ col1, row1 , ], dim=-1).flatten(0, 1).unsqueeze(1)
    h = self.transformer(pos + h.flatten(2).permute(2, 0, 1), self.query_pos.unsqueeze(1))
    return self.linear_class(h), self.linear_bbox(h).sigmoid()

In [ ]:
trans1 = transformer(hidden_dim=256)
detr1 = DETR(10, 256, 1, 1 ,1)

In [ ]:
someImages  = [trainset[0][0], trainset[1][0]]

writer.add_graph(trans1, trainset[0][0].unsqueeze(0))
writer.close()
#writer.add_graph(detr1, trainset[0][0].unsqueeze(0))
writer.close()

In [ ]:
trans1.eval()

out = trans1(trainset[0][0].unsqueeze(0))
